In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from sklearn.metrics import roc_auc_score

df = pd.read_csv ('given_dataset.csv')
df = df.drop('id', 1)
#print(df)

print()
corr_matrix = df.corr()
print(corr_matrix["T"].sort_values(ascending=False))
print()
#Normalization does not help
#normalized_df = (df - df.mean()) / df.std()
#normalized_df["T"] = df["T"]
#df = normalized_df

#Removing outliers through Z-Score does not help
#z_scores = stats.zscore(df)
#abs_z_scores = np.abs(z_scores)
#filtered_entries = (abs_z_scores < 3).all(axis=1)
#new_df = df[filtered_entries]
#df = new_df

print(df)
print()
print(df.describe())

In [ ]:
dataset = df.to_numpy()
features = dataset[:,:-1]
labels = dataset.T[-1]

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
features, labels = RandomUnderSampler().fit_resample(features, labels)

from imblearn.under_sampling import OneSidedSelection
features, labels = OneSidedSelection().fit_resample(features, labels)

In [ ]:
print(features)
print()
print(labels)

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_label, test_label = train_test_split(features, labels, test_size = 0.2, stratify = labels)
#Stratified split done to ensure there are enough instances of each class

print(train_data)
print()
print(train_label)

#from sklearn.tree import DecisionTreeClassifier
#dTree = DecisionTreeClassifier()
#dTree = dTree.fit(train_data, train_label)

In [ ]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'max_depth': [60, 80, 100],
#     'max_features': [4, 5, 6, 7, 8],
#     'n_estimators': [100, 300, 500]
# }

# rf = RandomForestRegressor()

# grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
# grid_search.fit(train_data, train_label)
# print(grid_search.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
#RandomForest as each tree in the forest is trained on a random subset of the data points with replacement (bagging / bootstrap aggregating)

clf = RandomForestRegressor(n_jobs = -1, max_depth=85, max_features=5, n_estimators=500)
clf.fit(train_data, train_label)
test_predict = clf.predict(test_data)
print(roc_auc_score(test_label, test_predict))

In [ ]:
# from xgboost import XGBRegressor
# clf = XGBRegressor(learning_rate=0.05, subsample=0.8, n_estimators=2500, n_jobs=-1, eval_metric='auc')
# clf.fit(train_data, train_label)
# test_predict = clf.predict(test_data)
# print(roc_auc_score(test_label, test_predict))

In [ ]:
#from imblearn.ensemble import BalancedRandomForestClassifier #If discrete is asked 

#clf = BalancedRandomForestClassifier()
#clf.fit(train_data, train_label)
#test_predict = clf.predict(test_data)
#print(roc_auc_score(test_label, test_predict))

In [ ]:
df2 = pd.read_csv ('to_predict.csv')
validation_id = df2['id']

df2 = df2.drop('id', 1)
validation_data = df2.to_numpy()


validation_predict_label = clf.predict(validation_data)


print(validation_predict_label)
print()
print(validation_predict_label.shape)
print()

In [ ]:
print(np.unique(validation_predict_label))

finalData = np.column_stack((validation_id, validation_predict_label))
print(finalData)

In [ ]:
output_df = pd.DataFrame(data = finalData, columns=['id','T'], dtype = np.float32)
output_df['id'] = output_df['id'].astype(int)
print(output_df)

In [ ]:
output_df.to_csv('predicted.csv', index=False)